In [ ]:
# %pip install openai

### Initialize OpenAI Client

In [1]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OPENAI_API_KEY and OPENAI_ORGANIZATION from .env file
api_key = os.getenv("OPENAI_API_KEY")
organization = os.getenv("OPENAI_ORGANIZATION")

In [2]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"), organization=os.getenv("OPENAI_ORGANIZATION")
)

### Set up the data structure for our conversation

In [3]:
# We want our messages to look something like this:
messages = [
    {
        "role": "system",
        "content": "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    },
    {"role": "user", "content": "Hello!"},
]

In [5]:
# So let's make some classes
from typing import List, Dict


class Message:
    def __init__(
        self,
        role: str,
        content: str,
        tool_calls: List = [],
        tool_call_id: str = None,
        name: str = None,
    ):
        self.role = role
        self.content = content
        self.tool_calls = tool_calls
        self.tool_call_id = tool_call_id
        self.name = name

    def to_dict(self):
        message_dict = {"role": self.role, "content": self.content}
        if self.tool_calls:
            message_dict["tool_calls"] = self.tool_calls
        if self.tool_call_id:
            message_dict["tool_call_id"] = self.tool_call_id
        if self.name:
            message_dict["name"] = self.name
        return message_dict


class Conversation:
    def __init__(self, messages: List[Dict[str, str]]):
        self.messages = [Message(**message) for message in messages]

    def __iter__(self):
        for message in self.messages:
            yield message

    def to_dict(self):
        return [message.to_dict() for message in self.messages]

    def add_message(self, message: Dict[str, str]):
        self.messages.append(Message(**message))

    def add_system_message(self, completion):
        self.add_message(
            {
                "role": completion.choices[0].message.role,
                "content": completion.choices[0].message.content,
                "tool_calls": completion.choices[0].message.tool_calls,
            }
        )

    def add_human_message(self, message):
        self.add_message(
            {
                "role": "user",
                "content": message,
            }
        )

    def add_tool_message(self, tool_called, response):
        self.add_message(
            {
                "role": "tool",
                "name": tool_called.function.name,
                "tool_call_id": tool_called.id,
                "content": str(response)
            }
        )

    def print_conversation(self):
        for message in self.messages:
            print(f"{message.role}: {message.content}")

In [6]:
# Create our conversation
conversation = Conversation(messages)
conversation.print_conversation()

system: You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user: Hello!


In [7]:
# And review our conversation so far from the eyes of OpenAI
conversation.to_dict()

[{'role': 'system',
  'content': "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user', 'content': 'Hello!'}]

### Now let's start chatting with OpenAI

In [8]:
# We'll call this a bunch, so let's make a function
def chat_with_openai(conversation):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=conversation.to_dict(),
    )

    return completion

In [9]:
# Let's call it now with our convesation
completion = chat_with_openai(conversation)

print(completion)

ChatCompletion(id='chatcmpl-8lN7lA8K6USKCy28Vraeue4qUwzHe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1706300997, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_b57c83dd65', usage=CompletionUsage(completion_tokens=9, prompt_tokens=46, total_tokens=55))


In [10]:
# What does just the message look like?
print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


In [11]:
# Let's add it to our convesation thread
conversation.add_system_message(completion)

# And let's review what these messages look like so far
conversation.print_conversation()

system: You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user: Hello!
assistant: Hello! How can I assist you today?


In [12]:
# Now let's add our new request
conversation.add_human_message("I want to add together 284, 317, and 42 and multiply the result by 13")
conversation.print_conversation()

system: You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user: Hello!
assistant: Hello! How can I assist you today?
user: I want to add together 284, 317, and 42 and multiply the result by 13


In [13]:
# Run the conversation again, and print the message
# (BTW, we expect an answer of 8359)
completion = chat_with_openai(conversation)
conversation.add_system_message(completion)
conversation.print_conversation()

system: You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user: Hello!
assistant: Hello! How can I assist you today?
user: I want to add together 284, 317, and 42 and multiply the result by 13
assistant: Sure! First, let's add 284, 317, and 42 together to get the total. Then, we can multiply that sum by 13. Let's start by finding the sum of 284, 317, and 42.


### We could repeat this process indefinitely

In [14]:
def run_conversation(new_message, conversation):
    conversation.add_human_message(new_message)
    completion = chat_with_openai(conversation)
    conversation.add_system_message(completion)
    conversation.print_conversation()
    return conversation

run_conversation("GO for it", conversation)

system: You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user: Hello!
assistant: Hello! How can I assist you today?
user: I want to add together 284, 317, and 42 and multiply the result by 13
assistant: Sure! First, let's add 284, 317, and 42 together to get the total. Then, we can multiply that sum by 13. Let's start by finding the sum of 284, 317, and 42.
user: GO for it
assistant: The sum of 284, 317, and 42 is 643. Now, let's multiply that sum by 13. 

643 * 13 = 8369

So, the result of adding 284, 317, and 42 together and then multiplying the sum by 13 is 8369.


### Now let's add in a Tool

In [15]:
def add_two_numbers(args):
    return args["number_a"] + args["number_b"]

In [16]:
# Let's specify the tools we want to include in our OpenAI request
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_two_numbers",
            "description": "Add two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "number_a": {
                        "type": "integer",
                        "description": "The first number",
                    },
                    "number_b": {
                        "type": "integer",
                        "description": "The second number.",
                    },
                },
                "required": ["number_a", "number_b"],
            },
        }
    }
]

In [17]:
# Let's add the tools to our chat request
def chat_with_openai_using_tools(conversation):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=conversation.to_dict(),
        tools=tools
    )

    return completion

In [18]:
# We want our messages to look something like this:
messages = [
    {
        "role": "system",
        "content": "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful",
    },
    {"role": "user", "content": "Help me add 13 and 27"},
]

conversation_with_tools = Conversation(messages)

In [19]:
conversation_with_tools.to_dict()

[{'role': 'system',
  'content': "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful"},
 {'role': 'user', 'content': 'Help me add 13 and 27'}]

In [20]:
# Let's call it now with our convesation
completion = chat_with_openai_using_tools(conversation_with_tools)
conversation_with_tools.add_system_message(completion)
print(completion.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_FnVzw616axLIBAylVzMffK9f', function=Function(arguments='{"number_a":13,"number_b":27}', name='add_two_numbers'), type='function')])


In [21]:
tool_called = completion.choices[0].message.tool_calls[0]
print(tool_called.function.name)
print(tool_called.function.arguments)
print(tool_called.id)

add_two_numbers
{"number_a":13,"number_b":27}
call_FnVzw616axLIBAylVzMffK9f


In [22]:
conversation_with_tools.to_dict()

[{'role': 'system',
  'content': "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful"},
 {'role': 'user', 'content': 'Help me add 13 and 27'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_FnVzw616axLIBAylVzMffK9f', function=Function(arguments='{"number_a":13,"number_b":27}', name='add_two_numbers'), type='function')]}]

In [23]:
# Let's call our amazing addition function
import json

response = add_two_numbers(json.loads(tool_called.function.arguments))
print(response)

40


In [24]:
conversation_with_tools.add_tool_message(tool_called, response)
conversation_with_tools.to_dict()

[{'role': 'system',
  'content': "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful"},
 {'role': 'user', 'content': 'Help me add 13 and 27'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_FnVzw616axLIBAylVzMffK9f', function=Function(arguments='{"number_a":13,"number_b":27}', name='add_two_numbers'), type='function')]},
 {'role': 'tool',
  'content': '40',
  'tool_call_id': 'call_FnVzw616axLIBAylVzMffK9f',
  'name': 'add_two_numbers'}]

In [25]:
# Let's call it now with our convesation
completion = chat_with_openai_using_tools(conversation_with_tools)
conversation_with_tools.add_system_message(completion)
conversation_with_tools.print_conversation()

system: You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful
user: Help me add 13 and 27
assistant: None
tool: 40
assistant: The sum of 13 and 27 is 40.


In [26]:
conversation_with_tools.to_dict()

[{'role': 'system',
  'content': "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful"},
 {'role': 'user', 'content': 'Help me add 13 and 27'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_FnVzw616axLIBAylVzMffK9f', function=Function(arguments='{"number_a":13,"number_b":27}', name='add_two_numbers'), type='function')]},
 {'role': 'tool',
  'content': '40',
  'tool_call_id': 'call_FnVzw616axLIBAylVzMffK9f',
  'name': 'add_two_numbers'},
 {'role': 'assistant', 'content': 'The sum of 13 and 27 is 40.'}]